<a href="https://colab.research.google.com/github/Aaryao9/meditranslate/blob/main/Meditranslate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# FULL SINGLE-CELL LoRA FINE-TUNING (EN → NE MEDICAL)
# ============================================================

# ---------- 0. INSTALL / RESET ENV ----------
!pip uninstall -y transformers peft accelerate sentence-transformers -q
!pip install transformers==4.41.2 peft==0.10.0 accelerate==0.29.3 datasets -q

# ---------- 1. IMPORTS ----------
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType
from shutil import make_archive

# ---------- 2. CONFIG ----------
MODEL_NAME = "rujengelal/my_awesome_english_to_nepali_tst"
DATASET_NAME = "Bibek-Poudel/ENG_NEP_MED_PARALLEL"
OUTPUT_DIR = "./eng-nep-med-lora"

MAX_SOURCE_LENGTH = 128
MAX_TARGET_LENGTH = 128

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------- 3. LOAD DATASET ----------
dataset = load_dataset(DATASET_NAME)
print(dataset)

# ---------- 4. TOKENIZER & MODEL ----------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# LoRA settings
model.config.decoder_start_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False  # REQUIRED for LoRA

# ---------- 5. PREPROCESS ----------
def preprocess_function(batch):
    inputs = batch["en"]
    targets = batch["ne"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_SOURCE_LENGTH,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        text_target=targets,
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding="max_length"
    )

    # Replace padding token ids in labels with -100
    labels["input_ids"] = [
        [(t if t != tokenizer.pad_token_id else -100) for t in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# ---------- 6. APPLY LoRA ----------
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "key", "value", "k_proj", "v_proj", "q_proj", "out_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ---------- 7. TRAINING ARGS ----------
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    fp16=torch.cuda.is_available(),
    save_steps=500,
    save_total_limit=3,
    logging_steps=100,
    report_to="none",
    remove_unused_columns=False
)

# ---------- 8. DATA COLLATOR ----------
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# ---------- 9. TRAIN ----------
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

# ---------- 10. SAVE MODEL ----------
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ LoRA fine-tuning finished and model saved to", OUTPUT_DIR)

# ---------- 11. ZIP MODEL FOR DOWNLOAD (COLAB/Temp Runtimes) ----------
make_archive("eng-nep-med-lora", 'zip', OUTPUT_DIR)
print("✅ Model zipped as eng-nep-med-lora.zip for download")

# ---------- 12. QUICK TEST ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

def translate_en_to_ne(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=4
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Quick test
print(translate_en_to_ne("The patient is suffering from high blood pressure."))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 139.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 125.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

...

trainable params: 2,359,296 || all params: 617,433,088 || trainable%: 0.38211363236820894


/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:469: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
100,3.077800
200,1.290400
300,1.116600
400,1.121500
500,1.026300
600,0.976600
700,1.007000
800,1.031400
900,0.986200
1000,0.922000


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in ver

✅ LoRA fine-tuning finished and model saved to ./eng-nep-med-lora
✅ Model zipped as eng-nep-med-lora.zip for download
बिरामीलाई उच्च रक्तचापको समस्या छ।
